In [ ]:
import torch
import torchvision
from torchvision import transforms
from torchvision.transforms import v2

In [2]:
DATASET_PATH = './knee-osteoarthritis'

In [3]:
TRAIN_PATH = f'{DATASET_PATH}/train'
VAL_PATH = f'{DATASET_PATH}/val'
TEST_PATH = f'{DATASET_PATH}/test'
AUTO_TEST_PATH = f'{DATASET_PATH}/auto_test'

In [4]:
transform_toTensor = transforms.Compose([transforms.ToTensor()])

In [5]:
train = torchvision.datasets.ImageFolder(TRAIN_PATH, transform_toTensor)
val = torchvision.datasets.ImageFolder(VAL_PATH, transform_toTensor)
# test = torchvision.datasets.ImageFolder(TEST_PATH, transform_toTensor)
# auto_test = torchvision.datasets.ImageFolder(AUTO_TEST_PATH, transform_toTensor)

In [ ]:
print(len(train))
print(len(val))
# print(len(test))
# print(len(auto_test))

### Augmentation 

In [7]:
import cv2
import numpy as np

#### Edges

In [8]:
sobel_y_1 = np.array([
    [-1, -1, -1],
    [0, 0, 0],
    [1, 1, 1],
])
sobel_y_2 = sobel_y_1 * -1

def getAugmentationEdges(image):
    grayscaled = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply two filters, upper and lower for different bones
    edges_1 = cv2.filter2D(grayscaled, -1, sobel_y_1)
    edges_2 = cv2.filter2D(grayscaled, -1, sobel_y_2)

    # Cutout noisy background 
    _, edges_1 = cv2.threshold(edges_1, 10, 255, cv2.THRESH_TOZERO)
    _, edges_2 = cv2.threshold(edges_2, 10, 255, cv2.THRESH_TOZERO)

    edges = edges_1 + edges_2

    max_brightness = edges.max()

    # Normalize color
    edges = cv2.convertScaleAbs(edges, alpha = 255/max_brightness, beta = 0)

    return edges

### Building Dataset

In [ ]:
example_img = train[0][0]
# print(example_img)
print(example_img.shape)

In [10]:
transform_baseImage = v2.Compose([
    v2.Resize(256),
    v2.CenterCrop(256),
    v2.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
])

transform_edgesImage = v2.Compose([
    v2.ToDtype(torch.float),
    # v2.Resize(256),
    # v2.CenterCrop(256),
    v2.Normalize(mean=[0.449], std=[0.226])
])

In [11]:

from torch.utils.data import Dataset, DataLoader

class KneeOsteoarthritis(Dataset):
    def __init__(self, dataset):
        self.images = []
        self.edges_images = []
        self.labels = []
        
        for data in dataset:
            image = data[0]
            image_agmentation = image.numpy()*255
            image_agmentation = np.moveaxis(image_agmentation, 0, -1)
            edges_image = getAugmentationEdges(image_agmentation)
            # print(image.shape, image_agmentation.shape, edges_image.shape)
            edges_image = torch.tensor(edges_image)
            label = data[1]
            
            image = transform_baseImage(image)
            # print(image.shape, edges_image.shape)
            edges_image = transform_edgesImage(edges_image.unsqueeze(0))
            
            self.images.append(image)
            self.edges_images.append(edges_image)            
            self.labels.append(label)
            
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        edges_image = self.edges_images[idx]
        label = self.labels[idx]
        
        return image, edges_image, label

In [ ]:
train_dataset = KneeOsteoarthritis(train)
val_dataset = KneeOsteoarthritis(val)
# test_dataset = KneeOsteoarthritis(test)
# auto_test_dataset = KneeOsteoarthritis(auto_test)

In [13]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.show()

In [ ]:
row = train_dataset[1]
normal_ex = row[0]
augmented_ex = row[1]
print(normal_ex.shape, augmented_ex.shape)


imshow(normal_ex)
imshow(augmented_ex)

### Configuring loader

In [15]:
from collections import Counter

In [16]:

def get_Class_Frequency(dataset):
  freq_table = dict(Counter(dataset.labels))
  return freq_table

def build_Sampler(dataset):
  freq_table = get_Class_Frequency(dataset)
  least_class_frequency = min(freq_table.values())

  normalized_freq = dict(freq_table)
  for i, val in enumerate(normalized_freq.values()):
    normalized_freq[i] = val / len(dataset.labels)
    
  print(normalized_freq, least_class_frequency)
  
  weights = np.zeros(len(dataset.labels))
  for i, weight in enumerate(weights):
    label = dataset.labels[i]
    weights[i] = 1 / freq_table[label]
      
  # print(weights)
  samples_weight = torch.from_numpy(weights)
  samples_weigth = samples_weight.double()
  sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight))
  
  return sampler

In [ ]:
train_sampler = build_Sampler(train_dataset)
val_sampler = build_Sampler(val_dataset)

In [18]:
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, sampler=train_sampler)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, sampler=val_sampler)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=True)

### Building Model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

In [20]:
import torch.nn as nn
import torch.nn.functional as F

from torchvision.models import resnet18, ResNet18_Weights

class AugmentedModel(nn.Module):
    def __init__(self, num_classes: int = 5, dropout: float = 0.5) -> None:
        super().__init__()
        
        # weights = ResNet18_Weights.DEFAULT
        # self.resnet18 = resnet18(weights=weights, progress=False)
        
        self.edgesClassifier = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.Dropout(p=dropout*0.4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=dropout*0.6),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout*0.8),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Flatten(),
            nn.Dropout(p=dropout),
            nn.Linear(32 * 6 * 6, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(64, num_classes),
            # nn.ReLU(inplace=True),
            # nn.Linear(64, num_classes),
        )
        
        self.baseClassifier = nn.Sequential(
            # nn.Dropout(p=dropout),
            nn.Linear(1000, 256),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=dropout),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, num_classes),
        )
        
        self.outputCombiner = nn.Sequential(
            nn.Linear(2 * num_classes, num_classes),
        )

    def forward(self, image: torch.Tensor, edges: torch.Tensor) -> torch.Tensor:
        
        out_edges = self.edgesClassifier(edges)
        # print(out_edges.shape)
        return out_edges
        # out_edges = self.edgesClassifier(edges)
      
        # out_image = self.resnet18(image)
        # out_image = self.baseClassifier(out_image)
        
        # concated = torch.cat((out_image, out_edges), 1)
        
        # res = self.outputCombiner(concated)
        # return res
      
net = AugmentedModel(3, 0.5)
net = net.to(device)

### Freeze Resnet

In [ ]:
# for param in net.resnet18.parameters():
#     param.requires_grad = False

In [ ]:
# print(sum(p.numel() for p in net.classifier.parameters()) ,sum(p.numel() for p in net.edgesClassifier.parameters()) )
print(sum(p.numel() for p in net.parameters()))

trainable_parameters = filter(lambda p: p.requires_grad, net.parameters())
print(sum(p.numel() for p in trainable_parameters))

### Setting up Model Training

In [ ]:
import torch.optim as optim
# criterion = nn.NLLLoss()

class_weights = get_Class_Frequency(train_dataset)
weights_tensor = torch.Tensor(list(class_weights.values())).to(device)
print(weights_tensor, weights_tensor.dtype)

criterion = nn.CrossEntropyLoss(weights_tensor)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [25]:
def get_lr(optimizer):
  for param_group in optimizer.param_groups:
    return param_group['lr']

In [26]:
def validate(model):
  with torch.no_grad():
    epoch_correct = 0
    epoch_samples = 0
    running_loss = 0.0
    
    for i, data in enumerate(val_loader):
      images, edges, labels = data
      images = images.to(device)
      edges = edges.to(device)
      labels = labels.to(device)
      
      # forward + backward + optimize
      outputs = model(images, edges)
      loss = criterion(outputs, labels)
      
      # Changing outputs (logits) to labels
      outputs_clear = outputs.max(1).indices
      
      epoch_correct += (outputs_clear == labels).float().sum()
      epoch_samples += len(outputs)
      
      running_loss += loss.item()
    
    # print(epoch_correct, epoch_samples)
    accuracy = epoch_correct / epoch_samples * 100
    
    return accuracy, running_loss / epoch_samples
    

In [27]:
from torchrl.record import TensorboardLogger

logger = TensorboardLogger(exp_name="my_exp")

In [28]:
epochCounter = 0

In [29]:
def train_many(model, epochs_nr, regularization_type = "L2", lambda_reg=0.01):
    global epochCounter
    for epoch in range(0, epochs_nr):  # loop over the dataset multiple times
        epoch_correct = 0
        epoch_samples = 0
        running_loss = 0.0
    
        for i, data in enumerate(train_loader, 0):

            # get the inputs; data is a list of [inputs, labels]
            images, edges, labels = data
            images = images.to(device)
            edges = edges.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(images, edges)
            loss = criterion(outputs, labels)
            
            # Apply L1 regularization
            if regularization_type == 'L1':
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss += lambda_reg * l1_norm
                
            # Apply L2 regularization
            elif regularization_type == 'L2':
                l2_norm = sum(p.pow(2).sum() for p in model.parameters())
                loss += lambda_reg * l2_norm
                
            loss.backward()
            
            optimizer.step()
            
            # Changing outputs (logits) to labels
            outputs_clear = outputs.max(1).indices
            
            epoch_correct += (outputs_clear == labels).float().sum()
            epoch_samples += len(outputs)
            
            running_loss += loss.item()
                
        tAccuracy = epoch_correct / epoch_samples * 100
        tLoss = running_loss / epoch_samples
        
        # Validation
        vAccuracy, vLoss = validate(model)
        
        logger.log_scalar("learning_rate", get_lr(optimizer), step=epochCounter)
        
        logger.log_scalar("train_acc", tAccuracy, step=epochCounter)
        logger.log_scalar("train_loss", tLoss, step=epochCounter)
        logger.log_scalar("val_acc", vAccuracy, step=epochCounter)
        logger.log_scalar("val_loss", vLoss, step=epochCounter)
        
        print(f'Epoch {epochCounter}: Training: accuracy: {tAccuracy:.3f}%, loss: {tLoss:.3f}; Validation: accuracy: {vAccuracy:.3f}%, loss: {vLoss:.3f}')
        
        epochCounter += 1
        
    print('Finished Training')

### Training Model

In [30]:
# %load_ext tensorboard

In [31]:
# %tensorboard --logdir logs/my_exp

In [ ]:
train_many(net, 25, 'L2', 0.01)

In [ ]:
train_many(net, 50, 'L2', 0.005)

In [ ]:
train_many(net, 75, 200, 'L2', 0.0025)